# Sympy tests

In [1]:
from sympy import symbols, cos, sin, pi, sqrt, simplify, pprint
from sympy.matrices import Matrix
import numpy as np

In [2]:
dtr = pi/180.0
rtd = 180.0/pi

In [39]:
# init_printing()
alpha0, alpha1, alpha2, alpha3, alpha4, alpha5, alpha6 = symbols('alpha0:7')
a0, a1, a2, a3, a4, a5, a6 = symbols('a0:7')
d1, d2, d3, d4, d5, d6, d7 = symbols('d1:8')
q1, q2, q3, q4, q5, q6, q7 = symbols('q1:8')

# DH Parameters
s = {alpha0: 0,     a0: 0,      d1: 0.75,  q1: q1,           # 1
     alpha1: -pi/2, a1: 0.35,   d2: 0,     q2: -pi/2 + q2,   # 2
     alpha2: 0,     a2: 1.25,   d3: 0,     q3: q3,           # 3
     alpha3: -pi/2, a3: -0.054, d4: 1.5,   q4: q4,           # 4
     alpha4: pi/2,  a4: 0,      d5: 0,     q5: q5,           # 5
     alpha5: -pi/2, a5: 0,      d6: 0,     q6: q6,           # 6
     alpha6: 0,     a6: 0,      d7: 0.303, q7: 0}            # G

def rot_x(q):
    T = Matrix([[1, 0, 0, 0],
                [0, cos(q), -sin(q), 0],
                [0, sin(q), cos(q), 0],
                [0, 0, 0, 1]])
    return T

def trans_x(d):
    T = Matrix([[1, 0, 0, d],
                [0, 1, 0, 0],
                [0, 0, 1, 0],
                [0, 0, 0, 1]])
    return T

def rot_z(q):
    T = Matrix([[cos(q), -sin(q), 0, 0],
                [sin(q), cos(q), 0, 0],
                [0, 0, 1, 0],
                [0, 0, 0, 1]])
    return T


def trans_z(d):
    T = Matrix([[1, 0, 0, 0],
                [0, 1, 0, 0],
                [0, 0, 1, d],
                [0, 0, 0, 1]])
    return T

def rot_y(q):
    T = Matrix([[cos(q),  0, sin(q), 0],
                [0,       1,      0, 0],
                [-sin(q), 0, cos(q), 0],
                [0,       0,      0, 1]])
    return T



def dh_trans(alpha0, a0, d1, q1):
    T = rot_x(alpha0) * trans_x(a0) * rot_z(q1) * trans_z(d1)
    return T

T0_1 = dh_trans(alpha0, a0, d1, q1)
T0_1 = T0_1.subs(s)

T1_2 = dh_trans(alpha1, a1, d2, q2)
T1_2 = T1_2.subs(s)

T2_3 = dh_trans(alpha2, a2, d3, q3)
T2_3 = T2_3.subs(s)

T3_4 = dh_trans(alpha3, a3, d4, q4)
T3_4 = T3_4.subs(s)

T4_5 = dh_trans(alpha4, a4, d5, q5)
T4_5 = T4_5.subs(s)

T5_6 = dh_trans(alpha5, a5, d6, q6)
T5_6 = T5_6.subs(s)

T6_G = dh_trans(alpha6, a6, d7, q7)
T6_G = T6_G.subs(s)



T0_2 = T0_1 * T1_2

T0_3 = T0_2 * T2_3

T0_4 = T0_3 * T3_4

T0_5 = T0_4 * T4_5

T0_6 = T0_5 * T5_6

T0_G = T0_6 * T6_G

R2_corr = rot_z(pi/2) * rot_x(pi/2)
R3_corr = R2_corr
R4_corr = rot_z(pi) * rot_y(-pi/2)
R5_corr = R2_corr
R6_corr = R4_corr
RG_corr = R4_corr

T0_2_corr = T0_2 * R2_corr

T0_3_corr = T0_3 * R3_corr

T0_4_corr = T0_4 * R4_corr

T0_5_corr = T0_5 * R5_corr

T0_6_corr = T0_6 * R6_corr

T0_G_corr = T0_G * RG_corr


# T1 = dh_trans(45*dtr, 5, 2, 30*dtr)
# T2 = rot_x(45*dtr) * trans_x(5) * rot_z(30*dtr) * trans_z(2)

# print('T0_1 =')
# pprint(T0_1)
# print('T1_2 =')
# pprint(T1_2)
# print('T0_2 =')
# pprint(T0_2)

print('T0_1 =')
# pprint(T0_1)
print('T0_1 =')
pprint(T0_1.evalf(subs={q1: 0, q2: 0, q3: 0, q4: 0, q5: 0, q6: 0}))

print('T1_2 =')
# pprint(T1_2)
print('T1_2 =')
pprint(T1_2.evalf(subs={q1: 0, q2: 0, q3: 0, q4: 0, q5: 0, q6: 0}))


print('T0_2 =')
# pprint(T0_2)
print('T0_2 =')
pprint(T0_2.evalf(subs={q1: 0, q2: 0, q3: 0, q4: 0, q5: 0, q6: 0}))


print('T3_4 =')
# pprint(T3_4)
print('T3_4 =')
pprint(T3_4.evalf(subs={q1: 0, q2: 0, q3: 0, q4: 0, q5: 0, q6: 0}))

print('T0_4 =')
# pprint(T0_4)
print('T0_4 =')
pprint(T0_4.evalf(subs={q1: 0, q2: 0, q3: 0, q4: 0, q5: 0, q6: 0}))


print('T4_5 =')
# pprint(T4_5)
print('T4_5 =')
pprint(T4_5.evalf(subs={q1: 0, q2: 0, q3: 0, q4: 0, q5: 0, q6: 0}))


print('T0_5 =')
# pprint(T0_5)
print('T0_5 =')
pprint(T0_5.evalf(subs={q1: 0, q2: 0, q3: 0, q4: 0, q5: 0, q6: 0}))

print('T5_6 =')
# pprint(T5_6)
print('T5_6 =')
pprint(T5_6.evalf(subs={q1: 0, q2: 0, q3: 0, q4: 0, q5: 0, q6: 0}))

print('T0_6 =')
# pprint(T0_6)
print('T0_6 =')
pprint(T0_6.evalf(subs={q1: 0, q2: 0, q3: 0, q4: 0, q5: 0, q6: 0}))


print('T6_G =')
# pprint(T6_G)
print('T6_G =')
pprint(T6_G.evalf(subs={q1: 0, q2: 0, q3: 0, q4: 0, q5: 0, q6: 0}))

print('T0_G =')
# pprint(T0_G)
print('T0_G =')
pprint(T0_G.evalf(subs={q1: 0, q2: 0, q3: 0, q4: 0, q5: 0, q6: 0}))






print('T0_2_corr =')
# pprint(T0_2_corr)
print('T0_2_corr =')
pprint(T0_2_corr.evalf(subs={q1: 0, q2: 0, q3: 0, q4: 0, q5: 0, q6: 0}))

print('T0_3_corr =')
# pprint(T0_3_corr)
print('T0_3_corr =')
pprint(T0_3_corr.evalf(subs={q1: 0, q2: 0, q3: 0, q4: 0, q5: 0, q6: 0}))

print('T0_4_corr =')
# pprint(T0_4_corr)
print('T0_4_corr =')
pprint(T0_4_corr.evalf(subs={q1: 0, q2: 0, q3: 0, q4: 0, q5: 0, q6: 0}))


print('T0_5_corr =')
# pprint(T0_5_corr)
print('T0_5_corr =')
pprint(T0_5_corr.evalf(subs={q1: 0, q2: 0, q3: 0, q4: 0, q5: 0, q6: 0}))

print('T0_6_corr =')
# pprint(T0_6_corr)
print('T0_6_corr =')
pprint(T0_6_corr.evalf(subs={q1: 0, q2: 0, q3: 0, q4: 0, q5: 0, q6: 0}))


print('T0_G_corr =')
# pprint(T0_G_corr)
print('T0_G_corr =')
pprint(T0_G_corr.evalf(subs={q1: 0, q2: 0, q3: 0, q4: 0, q5: 0, q6: 0}))




# print('T1 =', T1.evalf(3))
# print('T2 =', T2)



T0_1 =
T0_1 =
⎡1.0   0    0    0  ⎤
⎢                   ⎥
⎢ 0   1.0   0    0  ⎥
⎢                   ⎥
⎢ 0    0   1.0  0.75⎥
⎢                   ⎥
⎣ 0    0    0   1.0 ⎦
T1_2 =
T1_2 =
⎡ 0   1.0   0   0.35⎤
⎢                   ⎥
⎢ 0    0   1.0   0  ⎥
⎢                   ⎥
⎢1.0   0    0    0  ⎥
⎢                   ⎥
⎣ 0    0    0   1.0 ⎦
T0_2 =
T0_2 =
⎡ 0   1.0   0   0.35⎤
⎢                   ⎥
⎢ 0    0   1.0   0  ⎥
⎢                   ⎥
⎢1.0   0    0   0.75⎥
⎢                   ⎥
⎣ 0    0    0   1.0 ⎦
T3_4 =
T3_4 =
⎡1.0   0     0   -0.054⎤
⎢                      ⎥
⎢ 0    0    1.0   1.5  ⎥
⎢                      ⎥
⎢ 0   -1.0   0     0   ⎥
⎢                      ⎥
⎣ 0    0     0    1.0  ⎦
T0_4 =
T0_4 =
⎡ 0    0    1.0  1.85 ⎤
⎢                     ⎥
⎢ 0   -1.0   0     0  ⎥
⎢                     ⎥
⎢1.0   0     0   1.946⎥
⎢                     ⎥
⎣ 0    0     0    1.0 ⎦
T4_5 =
T4_5 =
⎡1.0   0    0     0 ⎤
⎢                   ⎥
⎢ 0    0   -1.0   0 ⎥
⎢                   ⎥
⎢ 0   1.0   0     0 ⎥
⎢

In [41]:
# Test Forward Kinematics
print('T0_G_corr =')
pprint(T0_G_corr.evalf(subs={q1: 10*dtr, q2: 0, q3: 0, q4: 0, q5: 0, q6: 0}))


T0_G_corr =
⎡0.984807753012208  -0.17364817766693   0   2.12029109223528 ⎤
⎢                                                            ⎥
⎢0.17364817766693   0.984807753012208   0   0.373864526516901⎥
⎢                                                            ⎥
⎢        0                  0          1.0        1.946      ⎥
⎢                                                            ⎥
⎣        0                  0           0          1.0       ⎦
